In [1]:
from ebooklib import epub
from bs4 import BeautifulSoup

import ebooklib

import pandas as pd

In [2]:
def extract_and_split_epub(file_path):
    # Load the EPUB file
    book = epub.read_epub(file_path)

    content_by_title = {}

    for item in book.items:
        # Check if the item is a chapter
        if item.get_type() == ebooklib.ITEM_DOCUMENT:
            # Parse the HTML content
            soup = BeautifulSoup(item.get_content(), "html.parser")

            # Find all titles (adjust the tags as per your EPUB structure)
            titles = soup.find_all(["h1", "h2", "h3"])  # Adjust tags if needed

            # Initialize current title
            current_title = "Untitled Section"
            content = ""

            for element in soup.find_all(True):  # Iterate over all elements
                if element.name in ["h1", "h2", "h3"]:
                    # Save previous section content
                    if current_title and content.strip():
                        content_by_title[current_title] = content.strip()

                    # Start a new section
                    current_title = element.get_text(strip=True)
                    content = ""
                else:
                    # Append content to the current section
                    content += element.get_text(strip=True) + "\n"

            # Save the last section
            if current_title and content.strip():
                content_by_title[current_title] = content.strip()

    return content_by_title

In [3]:
file_path = "../datasets/tableau-historique-paris/1.epub"
sections = extract_and_split_epub(file_path)

# Get only titles between AVERTISSEMENT and TABLE DES MATIÈRES.
chapters = []

start = False

for title, content in sections.items():
    if title == "AVERTISSEMENT.":
        start = True
        continue
    if not start:
        continue
    if title == "TABLE DES MATIÈRES.":
        break
    chapters.append({"title": title, "content": content})

/Users/mohameddjellal/Library/Caches/pypoetry/virtualenvs/rag-bench-aMooNfa2-py3.11/lib/python3.11/site-packages/ebooklib/epub.py:1395: UserWarning: In the future version we will turn default option ignore_ncx to True.
  warnings.warn('In the future version we will turn default option ignore_ncx to True.')
/Users/mohameddjellal/Library/Caches/pypoetry/virtualenvs/rag-bench-aMooNfa2-py3.11/lib/python3.11/site-packages/ebooklib/epub.py:1423: FutureWarning: This search incorrectly ignores the root element, and will be fixed in a future version.  If you rely on the current behaviour, change it to './/xmlns:rootfile[@media-type]'
  for root_file in tree.findall('//xmlns:rootfile[@media-type]', namespaces={'xmlns': NAMESPACES['CONTAINERNS']}):


In [4]:
## Get some statistics
df = pd.DataFrame.from_records(chapters)
df["length"] = df["content"].apply(lambda text: len(text.split()))

# Calculate statistics
total_sections = len(df)
average_length = df["length"].mean()
max_length = df["length"].max()
min_length = df["length"].min()
median_length = df["length"].median()
std_dev = df["length"].std()

In [5]:
chapters_stats = {
    "total_sections": total_sections,
    "average_length": average_length,
    "max_length": max_length,
    "min_length": min_length,
    "median_length": median_length,
    "std_dev": std_dev,
}

In [6]:
pd.DataFrame.from_dict(chapters_stats, orient="index", columns=["Value"]).T

,total_sections,average_length,max_length,min_length,median_length,std_dev
Value,66.0,1177.984848,6829.0,3.0,567.5,1558.89258
